# Evaluating Regression Lines Lab

### Introduction

In this lesson, let's put our knowledge about data science so far to the test.  And we'll do so with our real live movie data.

### Determining Quality

First, let's get some movies from the 538 dataset [provided here](https://raw.githubusercontent.com/fivethirtyeight/data/master/bechdel/movies.csv).

In [1]:
import pandas

def parse_file(fileName):
    movies_df = pandas.read_csv(fileName)
    return movies_df.to_dict('records')

movies = parse_file('https://raw.githubusercontent.com/fivethirtyeight/data/master/bechdel/movies.csv')


In [2]:
len(movies)

1794

And let's start looking at our data by examining the first entry in our dataset.

In [3]:
movies[0]

{'binary': 'FAIL',
 'budget': 13000000,
 'budget_2013$': 13000000,
 'clean_test': 'notalk',
 'code': '2013FAIL',
 'decade code': 1.0,
 'domgross': 25682380.0,
 'domgross_2013$': 25682380.0,
 'imdb': 'tt1711425',
 'intgross': 42195766.0,
 'intgross_2013$': 42195766.0,
 'period code': 1.0,
 'test': 'notalk',
 'title': '21 &amp; Over',
 'year': 2013}

So there, we can see what data is available about the movies in the dataset.  The `budget_2013$` seems to be the budget adjusted for inflation to be in 2013 dollars, and `domgross_2013$` seems to be the equivalent for domestic revenue, as `intgross_2013$` is the equivalent for 2013 international revenue.

Now, that first movie looks good, but there are some others that will not be so fun to play with.  Let's remove the values that have `nan`, which stands for not a number for their `domgross_2013`.  This is missing data, and there are only a few pieces of missing data here, so these are safe to remove without causing too much damage.  

In [500]:
import math
list(filter(lambda movie: math.isnan(movie['domgross_2013$']),movies))

[{'binary': 'FAIL',
  'budget': 19200000,
  'budget_2013$': 19200000,
  'clean_test': 'nowomen',
  'code': '2013FAIL',
  'decade code': 1.0,
  'domgross': nan,
  'domgross_2013$': nan,
  'imdb': 'tt2005374',
  'intgross': nan,
  'intgross_2013$': nan,
  'period code': 1.0,
  'test': 'nowomen-disagree',
  'title': 'The Frozen Ground',
  'year': 2013},
 {'binary': 'PASS',
  'budget': 4000000,
  'budget_2013$': 4142763,
  'clean_test': 'ok',
  'code': '2011PASS',
  'decade code': 1.0,
  'domgross': nan,
  'domgross_2013$': nan,
  'imdb': 'tt1422136',
  'intgross': 442550.0,
  'intgross_2013$': 458345.0,
  'period code': 1.0,
  'test': 'ok',
  'title': 'A Lonely Place to Die',
  'year': 2011},
 {'binary': 'PASS',
  'budget': 10000000,
  'budget_2013$': 10356908,
  'clean_test': 'ok',
  'code': '2011PASS',
  'decade code': 1.0,
  'domgross': nan,
  'domgross_2013$': nan,
  'imdb': 'tt1701990',
  'intgross': nan,
  'intgross_2013$': nan,
  'period code': 1.0,
  'test': 'ok',
  'title': 'Dete

Write a function called `remove_movies_missing_data` that returns the subset of movies that do not have `nan`.  To do so, you can import the math library and make use of the `math.isnan` method.

In [504]:
import math

def remove_movies_missing_data(movies):
    return [movie for movie in movies if not math.isnan(movie['domgross_2013$'])]

In [505]:
parsed_movies = remove_movies_missing_data(movies)
len(parsed_movies) # 1776
list(filter(lambda movie: math.isnan(movie['domgross_2013$']),parsed_movies)) # []


[]

To avoid working with such large numbers, let's divide both our budget and revenue numbers by 1 million.  It will make some of our future calculations easier.  It seems like the attributes that we can scale down are `budget`, `budget_2013$`, `domgross`, `domgross_2013$`, `intgross`, and `intgross_2013$`.

In [361]:
def scale_down_movie(movie):
    movie_copy = dict(movie)
    movie_copy.update({'budget': round(movie['budget']/1000000, 2), 'budget_2013$': round(movie['budget_2013$']/1000000, 2), 
                       'intgross': round(movie['intgross']/1000000, 2), 'intgross_2013$': round(movie['intgross_2013$']/1000000, 2),
                       'domgross': round(movie['domgross']/1000000, 2), 'domgross_2013$': round(movie['domgross_2013$']/1000000, 2)
                      })
    return movie_copy

scaled_movies = list(map(lambda movie: scale_down_movie(movie), parsed_movies))

Ok, now let's simply plot our dataset using Plotly to see how much money a movie makes domestically in 2013 dollars given a budget in 2013 dollars.  Set the text of the trace equal to a list of the movie titles, so that we can see which movie is associated with each point.

In [362]:
budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
domestic_revenues = list(map(lambda movie: movie['domgross_2013$'], scaled_movies))
titles = list(map(lambda movie: movie['title'], scaled_movies))

In [363]:
from graph import trace_values, plot
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
revenues_per_budgets = trace_values(budgets, domestic_revenues, text = titles)
plot([revenues_per_budgets])

Look at that datapoint was at the top well over the 1.5 billion mark.  What movie is that?

Write a function called highest_domestic_gross, that finds the highest grossing movie given a list of movies.

In [364]:
def highest_domestic_gross(movies):
    return max(movies, key=lambda movie: movie['domgross_2013$'])

In [365]:
highest_domestic_gross(scaled_movies)

{'binary': 'FAIL',
 'budget': 11.0,
 'budget_2013$': 42.27,
 'clean_test': 'notalk',
 'code': '1977FAIL',
 'decade code': nan,
 'domgross': 461.0,
 'domgross_2013$': 1771.68,
 'imdb': 'tt0076759',
 'intgross': 797.9,
 'intgross_2013$': 3066.45,
 'period code': nan,
 'test': 'notalk',
 'title': 'Star Wars',
 'year': 1977}

Huh, well we should've known.  Let's now zoom in on our dataset so that our plot no longer expands for just a few of the outliers.  Set the x-axis of our plot to go from zero to 300 million dollars, and the y-axis of our plot to go from zero to one billion dollars.

In [366]:
from graph import build_layout
revenues_per_budgets = trace_values(budgets, domestic_revenues, text = titles)
revenues_layout = build_layout(x_range = [0, 300], y_range = [0, 1000])
plot([revenues_per_budgets], revenues_layout)

Ok, well at least we now have a closer look at our data.  And we're still seeing Titanic up in the top right there. For a next step,

Ok, now imagine that we hired an outside consultant to predict revenue for us.  And the consultant told us the following: $R(x) = 1.5*budget + 10$ where $x$ is a movie's budget in 2013 dollars, and R(x) is the expected revenue in 2013 dollars.  Write a function called `outside_consultant_predicted_revenue` that provided a budget returns the expected revenue according to the outside consultant's formula. 

In [367]:
def outside_consultant_predicted_revenue(budget):
    return 1.5*budget + 10

Let's plot the consultant estimated revenue to see visually if his estimates line up.

In [368]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

from graph import trace_values, m_b_trace, plot
budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
domestic_revenues = list(map(lambda movie: movie['domgross_2013$'], scaled_movies))
titles = list(map(lambda movie: movie['title'], scaled_movies))

consultant_estimated_revenues = list(map(lambda budget: outside_consultant_predicted_revenue(budget),budgets))
movies_trace = trace_values(budgets, consultant_estimated_revenues, mode='line', name = 'consultant estimate')
plot([revenues_per_budgets, movies_trace], revenues_layout)

Overall, they don't look so bad.  Of course, we can calculate the RSS to place a number to how accurate his model really is.  Let's write a method called `error_for_consultant_model` which takes in a bugdet of a movie in our dataset, and returns the difference between the movie's gross domestic revenue in 2013 dollars, and the prediction by the consultant's model.

In [369]:
def error_for_consultant_model(movie):
    expected = outside_consultant_predicted_revenue(movie['budget_2013$'])
    return movie['budget_2013$'] - expected

In [370]:
american_hustle = scaled_movies[10]
error_for_consultant_model(american_hustle)

-30.0

Once haven written a formula that calculates the error for the consultant's model provided a budget, now write a method that calculates the RSS for the consulant's model.  When we move onto compare our consultant's model with others, we'll then have a metric to compare.

In [371]:
def rss_consultant(movies):
    return round(sum(list(map(lambda movie: error_for_consultant_model(movie)**2, movies))), 2)

In [372]:
rss_consultant(scaled_movies)

3900419.65

Ok, we'll find out if this number is any good later, but for right now let's just say that our RSS is good enough.  Use the derivative to write a function that provided a budget, returns the $\frac{\Delta R}{\Delta x}$ according to the consultant's model.  Remember that our consultant's model is $R(x) = 1.5x - 1000$ where $x$ is a budget, and $R(x)$ is an expected revenue.

In [373]:
def consultant_spend_a_little_more(budget):
    return 3

In [374]:
consultant_spend_a_little_more(100000000)

3

### A new model

Now imagine a data scientist in your company wants to take a crack at his own model for predicting a movie's revenue.  The data scientist notices, that in general, movies tend to make more money per year.

In [375]:
from graph import build_layout
years = list(map(lambda movie: movie['year'],movies))
years_and_revenues = trace_values(years, domestic_revenues, text = titles)
years_layout = build_layout(y_range = [0, 550])
plot([years_and_revenues], years_layout)

So the data scientist comes up with a new model, to indicate a movie's expected revenue is 1.5 million for every year after 1965 plus $1.4$ times the movie's budget.  Write a function called `revenue_with_year` that takes as arguments `budget` and `year` and returns expected revenue.  

In [403]:
def revenue_with_year(budget, year):
    return 1.1*budget + 1*(year - 1965)

Let's plot these estimated budgets side by side of the actual revenues and budgets.  

In [404]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

from graph import trace_values, m_b_trace, plot

budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
domestic_revenues = list(map(lambda movie: movie['domgross_2013$'], scaled_movies))
titles = list(map(lambda movie: movie['title'], scaled_movies))

consultant_estimated_revenues = list(map(lambda movie: revenue_with_year(movie['domgross_2013$'], movie['year']),scaled_movies))
movies_trace = trace_values(budgets, consultant_estimated_revenues, mode='markers', name = 'consultant estimate')
plot([revenues_per_budgets, movies_trace], revenues_layout)

As you can see, visually the model does fairly well.  Once again, let's find out how well.  Write a function called `rss_revenue_with_year` that returns the Residual Sum of Squares associated with the `revenue_with_year` model for the `scaled_movies` dataset.

In [405]:
def squared_error_revenue_with_year(movie):
    actual = movie['domgross_2013$']
    expected = revenue_with_year(movie['domgross_2013$'], movie['year'])
    return (actual - expected)**2

def rss_revenue_with_year(movies):
    squared_errors = list(map(lambda movie: squared_error_revenue_with_year(movie), movies))
    return round(sum(squared_errors), 2)

In [406]:
budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
domestic_revenues = list(map(lambda movie: movie['domgross_2013$'], scaled_movies))
titles = list(map(lambda movie: movie['title'], scaled_movies))

rss_revenue_with_year(scaled_movies) # 19957338.69

4241833.98

So the RSS here is $4,241,833.98$ as opposed to $3,900,419.65$ with our previous model.  So according to RSS it isn't better than the previous model.  Still, it's significantly worse to ignore completely.  

So using our knowledge of partial derivatives, let's write a function that returns the model's predicted increase in revenue from adding a movie debuting a year later. 

In [407]:
def years_and_revenue_increase_year():
    return 1

And using our knowledge of partial derivatives, let's write a function that returns the model's predicted increase of increasing the budget.

In [408]:
def years_and_revenue_increase_budget():
    return 1.1

### Our initial regression line, and improving upon it

Ok, so now that you have evaluated the model's of an outside consultant and an internal consultant, it's time to see if you can do any better.  Here we go.

We have our dataset, and should begin with an initial regression line by using our `build_regression_line` from earlier. 

In [410]:
from linear_equations import build_regression_line
budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
domestic_revenues = list(map(lambda movie: movie['domgross_2013$'], scaled_movies))
initial_regression_line = build_regression_line(budgets, domestic_revenues)
initial_regression_line

{'b': 0.5021166807532609, 'm': 1.788331924668964}

Using our `build_regression_line` method from earlier, we can get write `expected_revenue_per_budget`.  

In [412]:
def expected_revenue_per_budget(budget):
    return round(budget*initial_regression_line['m'] + initial_regression_line['b'], 2)

In [413]:
expected_revenue_per_budget(american_hustle['budget_2013$'])

72.04

Now remember that our `build_regression_line` formula is not very sophisticated.  It essentially draws a line between the point with the lowest $x$ value and the highest $x$.  Let's plot our line along our dataset to get a sense of how good a job this does.

In [436]:


budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
estimated_revenues = list(map(lambda budget: expected_revenue_per_budget(budget), budgets))
initial_regression_trace = trace_values(budgets, estimated_revenues, mode = 'line')
plot([movies_trace, initial_regression_trace])

By now you should be able to guess our next step: put a number to how well this line matches our data.  We'll write a function called `regression_revenue_error` that provided a movie and an `m` and `b` value of a regression line, returns the difference between our `initial_regression_lines`'s expected revenue and the actual revenue error.

In [414]:
def regression_revenue_error(m, b, movie):
    expected = (m*movie['budget_2013$'] + b)
    actual = movie['domgross_2013$']
    return actual - expected

In [517]:
regression_revenue_error(initial_regression_line['m'], initial_regression_line['b'], movies[10]) # 76897630.51112476

76897630.51112476

Ok, now plot the cost curve from changing values of $m$ from $1.0$ to $2.0$.  We don't ask you to write a function for calculating the RSS, as you already wrote one in the `error` library which is available to you.

In [419]:
from error import residual_sum_squares
residual_sum_squares(budgets, domestic_revenues, initial_regression_line['m'], initial_regression_line['b'])

24163798.64890295

In [437]:
from graph import trace_values
large_m_range = list(range(10, 20))
m_range = list(map(lambda m_value: m_value/10,large_m_range))
cost_values = list(map(lambda m_value: residual_sum_squares(budgets, domestic_revenues, m_value, initial_regression_line['b']),m_range))
rss_trace = trace_values(x_values=m_range, y_values=cost_values, mode = 'line')

In [438]:
plot([rss_trace])

Ok, so based on this, it appears that with our $b = 0.5021166807532609$, the slope of our regression line that produces the lowest error is between $1.3$ and $1.4$.  In fact if we replace our initial line value of $m$ to $1.3$ we see that our RSS does in fact from a previous value of $24,163,798$.

In [432]:
residual_sum_squares(budgets, domestic_revenues, 1.3, initial_regression_line['b'])

22065937.75716766

### Changing multiple variables

Ok, now it's time for us to not just alter a variable like the slope of our regression, but to find the 'best fit regression line'.  As you know, the technique for that is to use gradient descent.

Remember that we derived our gradient formulas equal from our cost curve: 

$$J(m,b) = \sum_{i = 1}^n(y_i - (mx_i + b))^2 $$

We derived the gradient of our cost function as the following: 

$$ \frac{dJ}{db}J(m,b) = -2\sum_{i = 1}^n(y_i - (mx_i + b)) = -2\sum_{i = 1}^n \epsilon_i $$
$$ \frac{dJ}{dm}J(m,b) = -2\sum_{i = 1}^n x(y_i - (mx_i + b)) = -2\sum_{i = 1}^n x_i*\epsilon_i$$


#### remove the 2, to simplify

Now looking at our top function $\frac{dJ}{dm}$, we see that it equals to negative 2, multiplied by the sum of the errors for a a provided $m$ and $b$ values and a dataset.  And lucky, for us we already have a function called `regression_revenue_error` that returns the error at a given point, provided our $m$ and $b$ values.

Your task is two write a function called `b_gradient` that takes in values of $m$, $b$ and our (scaled) movies, and returns the $b gradient$, -2 time the sum of the errors for the dataset.

In [454]:
def b_gradient(m, b, movies):
    n = len(movies)
    errors = list(map(lambda movie: regression_revenue_error(m, b, movie), movies))
    return -1 * sum(errors)/n

In [455]:
b_gradient(1.79, 0.50, scaled_movies)

5.367987612612615

In [456]:
def m_gradient(m, b, movies):
    n = len(movies)
    errors_times_x = list(map(lambda movie: regression_revenue_error(m, b, movie)*movie['budget_2013$'], movies))
    return -1 * sum(errors_times_x)/n

In [457]:
m_gradient(1.79, 0.50, scaled_movies)

2520.578267701572

Ok, now we just wrote two functions that tell us how to update the corresponding values of $m$ and $b$.  Our next step is to write a function called `step_gradient` that will use these functions to take the step down along our cost curve.

Remember that with each step we want to move our `current_b` value in the negative direction of calculated `b_gradient`, and want to move our `current_m` value in the negative direction of calculated `m_gradient`.  

`current_m` = `old_m` $ -  \alpha(-2*\sum_{i=1}^n x_i*\epsilon_i )$

`current_b` =  `old_b` $ - \alpha( -2*\sum_{i=1}^n \epsilon_i )$

The `step_gradient` function would take as arguments the `b_current`, `m_current`, and list of scaled movies and a learning rate, and return a new calculated `b_current` and `m_current` with a dictionary of keys `b` and `m` that point to the current values.   

In [458]:
def step_gradient(b_current, m_current, movies, learning_rate):
    b_change = b_gradient(m_current, b_current, movies)
    m_change = m_gradient(m_current, b_current, movies) 
    new_b = b_current - (learning_rate * b_change)
    new_m = m_current - (learning_rate * m_change)
    return {'b': new_b, 'm': new_m}

Now let's plot the first 10 steps of gradient descent.

In [459]:
initial_regression_line # {'b': 0.5021166807532609, 'm': 1.788331924668964}

{'b': 0.5021166807532609, 'm': 1.788331924668964}

Then let's see how our formula changes over time using gradient descent.

In [460]:
step_gradient(initial_regression_line['b'], initial_regression_line['m'], scaled_movies, .0001)

{'b': 0.5015889931744079, 'm': 1.537287989729067}

Now write a function that returns a set of 10 iterations.

In [476]:
# set our initial step with m and b values, and the corresponding error.
def generate_steps(m, b, number_of_steps, movies):
    iterations = []
    for i in range(number_of_steps):
        iteration = step_gradient(b, m, movies, .0001)
        # {'b': value, 'm': value}
        b = iteration['b']
        m = iteration['m']
        # update values of b and m
        iterations.append(iteration)
    return iterations

In [493]:
iterations = generate_steps(initial_regression_line['m'], initial_regression_line['b'], 100, scaled_movies)

And we can see how this changes over time.

In [494]:
def to_line(m, b):
    initial_x = 0
    ending_x = 500
    initial_y = m*initial_x + b
    ending_y = m*ending_x + b
    return {'data': [{'x': [initial_x, ending_x], 'y': [initial_y, ending_y]}]}

frames = list(map(lambda iteration: to_line(iteration['m'], iteration['b']),iterations))
frames[0]

{'data': [{'x': [0, 500], 'y': [0.5015889931744079, 769.145583857708]}]}

In [489]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

init_notebook_mode(connected=True)

budgets = list(map(lambda movie: movie['budget_2013$'], scaled_movies))
domestic_revenues = list(map(lambda movie: movie['domgross_2013$'], scaled_movies))

figure = {'data': [{'x': [0], 'y': [0]}, {'x': budgets, 'y': domestic_revenues, 'mode': 'markers'}],
          'layout': {'title': 'Regression Line',
                     'updatemenus': [{'type': 'buttons',
                                      'buttons': [{'label': 'Play',
                                                   'method': 'animate',
                                                   'args': [None]}]}]
                    },
          'frames': frames}
iplot(figure)

Finally, let's calculate the RSS associated with our formula as opposed to the other.

In [495]:
iterations[-1] # {'b': 0.5509918331403825, 'm': 1.3796546010160367}

{'b': 0.6733901508952449, 'm': 1.378542026072047}

In [496]:
residual_sum_squares(budgets, domestic_revenues, iterations[-1]['m'], iterations[-1]['b'])

22043348.930944957

Using this last iteration, we have an RSS $22043348$, better than all previous models - and we have the data, and knowledge to prove it.  Nice work!